In [4]:
#imports
import numpy as np
import pandas as pd
import os
import re
import string
import time
import matplotlib.pyplot as plt
import seaborn as sns
import pycountry

In [9]:
pycountry.subdivision.get(code = f'US-CA').country.name

AttributeError: module 'pycountry' has no attribute 'subdivision'